##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# 모델 저장과 복원

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/keras/save_and_load"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a></td>
</table>

모델 진행 상황은 훈련 중 및 훈련 후에 저장할 수 있습니다. 즉, 모델이 중단된 위치에서 다시 시작하고 긴 훈련 시간을 피할 수 있습니다. 저장은 또한 모델을 공유할 수 있고 다른 사람들이 작업을 다시 만들 수 있음을 의미합니다. 연구 모델 및 기술을 게시할 때 대부분의 머신러닝 실무자는 다음을 공유합니다.

- 모델을 만드는 코드
- 모델의 훈련된 가중치 또는 파라미터

이런 데이터를 공유하면 다른 사람들이 모델의 작동 방식을 이해하고 새로운 데이터로 모델을 실험하는데 도움이 됩니다.

주의: TensorFlow 모델은 코드이며 신뢰할 수 없는 코드에 주의하는 것이 중요합니다. 자세한 내용은 [TensorFlow 안전하게 사용하기](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md)를 참조하세요.

### 저장 방식

사용 중인 API에 따라 다양한 방법으로 TensorFlow 모델을 저장할 수 있습니다. 이 가이드에서는 TensorFlow에서 모델을 빌드하고 훈련하는 고급 API인 [tf.keras](https://www.tensorflow.org/guide/keras)를 사용합니다. 이 튜토리얼에서 사용된 새로운 고수준 `.keras` 형식은 저수준 또는 레거시 형식보다 디버깅하기 쉬운 강력하고 효율적인 이름 기반 저장을 제공하기 때문에 Keras 객체를 저장하는 데 권장됩니다. 고급 저장 또는 직렬화 워크플로, 특히 사용자 정의 객체와 관련된 워크플로는 [Keras 모델 저장 및 로드 가이드](https://www.tensorflow.org/guide/keras/save_and_serialize)를 참조하세요. 다른 접근 방식은 [SavedModel 형식 가이드 사용하기](../../guide/saved_model.ipynb)를 참조하세요.

## 설정

### 설치와 임포트

필요한 라이브러리를 설치하고 텐서플로를 임포트(import)합니다:

In [3]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [4]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.15.0


### 예제 데이터셋 받기

[MNIST 데이터셋](http://yann.lecun.com/exdb/mnist/)으로 모델을 훈련하여 가중치를 저장하는 예제를 만들어 보겠습니다. 모델 실행 속도를 빠르게 하기 위해 샘플에서 처음 1,000개만 사용겠습니다:

In [5]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11490434/11490434 [==============================] - 0s 0us/step


### 모델 정의

먼저 간단한 모델을 하나 만들어 보죠.

In [6]:
# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## 훈련하는 동안 체크포인트 저장하기

훈련된 모델을 다시 훈련할 필요 없이 사용하거나 훈련 과정이 중단된 경우 중단한 부분에서 훈련을 다시 시작할 수 있습니다. `tf.keras.callbacks.ModelCheckpoint` 콜백을 사용하면 훈련 *도중* 또는 훈련 *종료 시* 모델을 지속적으로 저장할 수 있습니다.

### 체크포인트 콜백 사용하기

훈련하는 동안 가중치를 저장하기 위해 `ModelCheckpoint` 콜백을 만들어 보죠:

In [7]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
32/32 [==============================] - ETA: 0s - loss: 1.1296 - sparse_categorical_accuracy: 0.6950
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 5s 27ms/step - loss: 1.1296 - sparse_categorical_accuracy: 0.6950 - val_loss: 0.7032 - val_sparse_categorical_accuracy: 0.7830
Epoch 2/10
30/32 [===========================>..] - ETA: 0s - loss: 0.4143 - sparse_categorical_accuracy: 0.8771
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 13ms/step - loss: 0.4232 - sparse_categorical_accuracy: 0.8720 - val_loss: 0.5429 - val_sparse_categorical_accuracy: 0.8300
Epoch 3/10
26/32 [=======================>......] - ETA: 0s - loss: 0.2842 - sparse_categorical_accuracy: 0.9327
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 12ms/step - loss: 0.2811 - sparse_categorical_accuracy: 0.9350 - val_loss: 0.4706 - val_sparse_categorical_accuracy: 0.8480
Epoch 4/10
26/32 [=======

이 코드는 텐서플로 체크포인트 파일을 만들고 에포크가 종료될 때마다 업데이트합니다:

In [8]:
os.listdir(checkpoint_dir)

['cp.ckpt.index', 'checkpoint', 'cp.ckpt.data-00000-of-00001']

두 모델이 동일한 아키텍처를 공유하기만 한다면 두 모델 간에 가중치를 공유할 수 있습니다. 따라서 가중치 전용에서 모델을 복원할 때 원래 모델과 동일한 아키텍처로 모델을 만든 다음 가중치를 설정합니다.

이제 훈련되지 않은 새로운 모델을 다시 빌드하고 테스트 세트에서 평가합니다. 훈련되지 않은 모델은 확률 수준(~10% 정확도)에서 수행됩니다.

In [9]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3916 - sparse_categorical_accuracy: 0.0650 - 165ms/epoch - 5ms/step
Untrained model, accuracy:  6.50%


체크포인트에서 가중치를 로드하고 다시 평가해 보죠:

In [10]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4224 - sparse_categorical_accuracy: 0.8670 - 71ms/epoch - 2ms/step
Restored model, accuracy: 86.70%


### 체크포인트 콜백 매개변수

이 콜백 함수는 몇 가지 매개변수를 제공합니다. 체크포인트 이름을 고유하게 만들거나 체크포인트 주기를 조정할 수 있습니다.

새로운 모델을 훈련하고 다섯 번의 에포크마다 고유한 이름으로 체크포인트를 저장해 보겠습니다:

In [11]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Calculate the number of batches per epoch
import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)    # round up the number of batches to the nearest whole integer

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*n_batches)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=50,
          batch_size=batch_size,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


이제 결과로 나온 체크포인트를 검토하고 최신 체크포인트를 선택합니다.

In [12]:
os.listdir(checkpoint_dir)

['cp-0015.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0050.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index']

In [13]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

참고: 기본 TensorFlow 형식은 가장 최근의 체크포인트 5개만 저장합니다.

테스트를 위해 모델을 재설정하고 최신 체크포인트를 로드합니다.

In [14]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4911 - sparse_categorical_accuracy: 0.8790 - 154ms/epoch - 5ms/step
Restored model, accuracy: 87.90%


## 이 파일들은 무엇인가요?

위의 코드는 이진 형식의 훈련된 가중치만 포함하는 [체크포인트](../../guide/checkpoint.ipynb) 형식의 파일 모음에 가중치를 저장합니다. 체크포인트에는 다음이 포함됩니다.

- 모델의 가중치를 포함하는 하나 이상의 샤드
- 어떤 가중치가 어떤 샤드에 저장되어 있는지 나타내는 인덱스 파일

단일 머신에서 모델을 훈련하는 경우 접미사가 `.data-00000-of-00001`인 하나의 샤드를 갖게 됩니다.

## 수동으로 가중치 저장하기

가중치를 수동으로 저장하려면 `tf.keras.Model.save_weights`를 사용합니다. 기본적으로 `tf.keras`, 그리고 특히 `Model.save_weights` 메서드는 <code>.ckpt</code> 확장자가 있는 TensorFlow <a>체크포인트</a> 형식을 사용합니다. `.h5` 확장자를 사용하여 HDF5 형식으로 저장하려면 [모델 저장 및 로드](https://www.tensorflow.org/guide/keras/save_and_serialize) 가이드를 참조하세요.

In [15]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4911 - sparse_categorical_accuracy: 0.8790 - 187ms/epoch - 6ms/step
Restored model, accuracy: 87.90%


## 전체 모델 저장하기

`tf.keras.Model.save`를 호출하여 모델의 아키텍처, 가중치 및 훈련 구성을 단일 `model.keras` zip 아카이브에 저장합니다.

전체 모델을 세 가지 다른 파일 형식(새로운 `.keras` 형식 및 두 가지 레거시 형식: `SavedModel` 및 `HDF5`)으로 저장할 수 있습니다. 모델을 `path/to/model.keras`로 저장하면 자동으로 최신 형식으로 저장됩니다.

**참고:** Keras 객체의 경우 디버깅하기 쉬운 더 풍부한 이름 기반 저장과 재로드를 위해 새로운 고수준 `.keras` 형식을 사용하는 것이 좋습니다. 기존 코드 형식에는 저수준 SavedModel 형식과 레거시 H5 형식이 계속 지원됩니다.

다음을 통해 SavedModel 형식으로 전환할 수 있습니다.

- `save()`로 `save_format='tf'` 전달
- 확장자 없이 파일 이름 전달

다음을 통해 H5 형식으로 전환할 수 있습니다.

- `save()`로 `save_format='h5'` 전달
- `.h5`로 끝나는 파일 이름 전달

Saving a fully-functional model is very useful—you can load them in TensorFlow.js ([Saved Model](https://www.tensorflow.org/js/tutorials/conversion/import_saved_model), [HDF5](https://www.tensorflow.org/js/tutorials/conversion/import_keras)) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite ([Saved Model](https://www.tensorflow.org/lite/models/convert/#convert_a_savedmodel_recommended_), [HDF5](https://www.tensorflow.org/lite/models/convert/#convert_a_keras_model_))

*Custom objects (for example, subclassed models or layers) require special attention when saving and loading. Refer to the **Saving custom objects** section below.

### 새로운 고수준 `.keras` 형식

`.keras` 확장자로 표시되는 새로운 Keras v3 저장 형식은 이름 기반 저장을 구현하는 보다 간단하고 효율적인 형식으로, Python의 관점에서 로드한 내용이 저장한 내용과 정확히 일치하는지 확인합니다. 이렇게 하면 디버깅이 훨씬 쉬워지며, Keras에 권장되는 형식입니다.

아래 섹션에서는 모델을 `.keras` 형식으로 저장하고 복원하는 방법을 설명합니다.

In [16]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a `.keras` zip archive.
model.save('my_model.keras')

Epoch 1/5
32/32 [==============================] - 1s 3ms/step - loss: 1.1741 - sparse_categorical_accuracy: 0.6570
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4433 - sparse_categorical_accuracy: 0.8700
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2858 - sparse_categorical_accuracy: 0.9290
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2183 - sparse_categorical_accuracy: 0.9490
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1669 - sparse_categorical_accuracy: 0.9550


`.keras` zip 아카이브에서 새 Keras 모델을 다시 로드합니다.

In [17]:
new_model = tf.keras.models.load_model('my_model.keras')

# Show the model architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


로드된 모델로 평가 및 예측을 실행해봅니다.

In [18]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4493 - sparse_categorical_accuracy: 0.8570 - 238ms/epoch - 7ms/step
Restored model, accuracy: 85.70%
32/32 [==============================] - 0s 2ms/step
(1000, 10)


### SavedModel 포맷

SavedModel 형식은 모델을 직렬화하는 또 다른 방법입니다. 이 형식으로 저장된 모델은 `tf.keras.models.load_model`을 사용하여 복원할 수 있으며 TensorFlow Serving과 호환됩니다. [SavedModel 가이드](../../guide/saved_model.ipynb)에 SavedModel을 제공/검사하는 방법이 자세히 설명되어 있습니다. 아래 섹션은 모델을 저장하고 복원하는 단계를 보여줍니다.

In [19]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 4ms/step - loss: 1.1526 - sparse_categorical_accuracy: 0.6670
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4334 - sparse_categorical_accuracy: 0.8800
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2781 - sparse_categorical_accuracy: 0.9250
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2014 - sparse_categorical_accuracy: 0.9570
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1618 - sparse_categorical_accuracy: 0.9610


SavedModel 형식은 protobuf 바이너리와 TensorFlow 체크포인트를 포함하는 디렉토리입니다. 저장된 모델 디렉토리를 검사합니다.

In [20]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model
assets	fingerprint.pb	keras_metadata.pb  saved_model.pb  variables


저장된 모델로부터 새로운 케라스 모델을 로드합니다:

In [21]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


복원된 모델은 원본 모델과 동일한 매개변수로 컴파일되어 있습니다. 이 모델을 평가하고 예측에 사용해 보죠:

In [22]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4489 - sparse_categorical_accuracy: 0.8500 - 169ms/epoch - 5ms/step
Restored model, accuracy: 85.00%
32/32 [==============================] - 0s 2ms/step
(1000, 10)


### HDF5 파일로 저장하기

Keras는 [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) 표준을 따르는 기본 레거시 고수준 저장 형식을 제공합니다.

In [23]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 3ms/step - loss: 1.1635 - sparse_categorical_accuracy: 0.6820
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4291 - sparse_categorical_accuracy: 0.8820
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2926 - sparse_categorical_accuracy: 0.9250
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2222 - sparse_categorical_accuracy: 0.9370
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1553 - sparse_categorical_accuracy: 0.9670


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


이제 이 파일로부터 모델을 다시 만들어 보죠:

In [24]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


정확도를 확인해 보겠습니다:

In [25]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4304 - sparse_categorical_accuracy: 0.8580 - 160ms/epoch - 5ms/step
Restored model, accuracy: 85.80%


Keras는 아키텍처를 검사하여 모델을 저장합니다. 이 기술은 모든 내용을 저장합니다.

- 가중치 값
- 모델 구조
- 모델의 훈련 구성(`.compile()` 메서드에 전달하는 내용)
- 존재하는 옵티마이저와 그 상태(훈련을 중단한 곳에서 다시 시작할 수 있게 해줌)

체크포인트가 호환되지 않기 때문에 케라스는 v1.x 옵티마이저(`tf.compat.v1.train`)를 저장할 수 없습니다. v1.x 옵티마이저를 사용하려면 로드한 후에 모델을 다시 컴파일해야 합니다. 따라서 옵티마이저의 상태를 잃게 됩니다.


### 사용자 정의 객체

SavedModel 형식을 사용하는 경우 이 섹션을 건너뛸 수 있습니다. 고수준 `.keras`/HDF5 형식과 저수준 SavedModel 형식의 주요 차이점은 `.keras`/HDF5 형식이 객체 구성을 사용하여 모델 아키텍처를 저장하고 SavedModel은 실행 그래프를 저장한다는 것입니다.따라서 SavedModels는 원본 코드가 없어도 하위 클래스 모델 및 사용자 정의 레이어와 같은 사용자 정의 객체를 저장할 수 있습니다. 단, 결과적으로 저수준의 SavedModels를 디버깅하는 것이 더 어려울 수 있으므로 이름 기반의 Keras 네이티브 특성을 가진 고수준의 `.keras` 형식을 대신 사용하는 것이 좋습니다.

사용자 정의 개체를 `.keras` 및 HDF5에 저장하려면 다음을 수행해야 합니다.

1. 이 객체에 `get_config` 메서드를 정의하고 선택적으로 `from_config` 클래스 메서드를 정의합니다.
    - `get_config(self)`는 객체를 다시 생성하기 위해 필요한 JSON 직렬화된 매개변수 딕셔너리를 반환합니다.
    - `from_config(cls, config)`는 `get_config`에서 반환된 설정을 사용해 새로운 객체를 만듭니다. 기본적으로 이 함수는 이 설정을 초기화 메서드의 매개변수로 사용합니다(`return cls(**config)`).
2. 다음 세 가지 방법 중 하나를 사용하여 사용자 정의 객체를 모델에 전달합니다.
    - `@tf.keras.utils.register_keras_serializable` 데코레이터를 사용하여 사용자 정의 객체를 등록합니다. **(권장 사항)**
    - 모델을 로드할 때 `custom_objects` 인수에 객체를 직접 전달합니다. 인수는 문자열 클래스 이름을 파이썬 클래스에 매핑하는 사전이어야 합니다. 예: `tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})`
    - `custom_objects` 사전 인수에 포함된 객체와 함께 `tf.keras.utils.custom_object_scope`를 사용하고 범위 안에 `tf.keras.models.load_model(path)` 호출을 배치합니다.

사용자 정의 객체 및 <code>get_config</code>의 예를 보려면 [처음부터 레이어 및 모델 작성하기](https://www.tensorflow.org/guide/keras/custom_layers_and_models) 튜토리얼을 참조하세요.
